#This notebook can be divided to three parts.
1. Openmm installation and a test simulation
2. Openmm simulation with Amber input, i.e., a prmtop file, and a (inp)crd file
3. Openmm simulation with VMD/Charmm input, i.e., a psf file and a pdb file.

Note: The method of how to generate Amber or VMD/Charmm input is not incluede here, you are supposed to be confident with at least one of them before using this notebook.

Created by quantaosun@gmail.com, based on the official openmm documentation in https://openmm.org/documentation 

A potential update would include method of standard free energy of protein ligand binding.

## Note, the next 3htb, example, i.e., the first OpenMM simulation is just fro testing purpose, to verify our simulation software is working. For a more proper simulation you should use the following two, i.e., the Amber input version or the VMD/Charmm input verion.

In [ ]:
#@title  Install Dependencies
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
#! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
#@title Install OpenMM
!yes|conda install -c conda-forge openmm

In [ ]:
#@title Install Pdbfixer, you could skip this step
!yes|conda install -c conda-forge pdbfixer

In [ ]:
#@title Verify OpenMM installation works or not.
!python -m openmm.testInstallation

In [ ]:
#@title Prepare the raw PDB structure with PDBfixer
!pdbfixer --pdbid=3htb --output=myfile.pdb --add-residues 
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
fixer = PDBFixer(filename='myfile.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(fixer.topology.getUnitCellDimensions())
PDBFile.writeFile(fixer.topology, fixer.positions, open('input.pdb', 'w'))

In [ ]:
#@title This is not a proper simulation, just for testing purpose
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

pdb = PDBFile('input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
simulation.step(100000)

#This is an example of using Amber input, this could be a proper simulation, as long as your Amber input is good.✈

In [ ]:
#@title This is the code example taken from Openmm's user guide document.

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

prmtop = AmberPrmtopFile('SYS_gaff2.prmtop')
inpcrd = AmberInpcrdFile('SYS_gaff2.crd')
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometer,
        constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('amber_output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

#This is an example of using VMD/Charmm input. This is my personal favourate, since I use VMD exclusively.✈⛹ Just be reminded the topolog and parameter stuff sometime easy to go wrong.

In [ ]:
#@title 6 👋Download Charmm Force Field
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/top_all36_prot.prm 
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/top_all36_prot.rtf 
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/toppar_water_ions.mod.str
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/toppar_water_ions.str

In [ ]:
#@title Be ware of topology of small molecule. Upload all the rtf and prm you would run a normal NAMD simulation, to here in order to run an OpenMM simulation.

In [ ]:
#@title  This is a simulation with PSF and PDB files generated from VMD, as an input for OpenMM
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout, exit, stderr

psf = CharmmPsfFile('ionized.psf')
pdb = PDBFile('ionized.pdb')
params = CharmmParameterSet('top_all36_prot.rtf', 'top_all36_prot.prm','toppar_water_ions.mod.str','toppar_water_ions.str','ligand_ligpargen.prm','ligand_ligpargen.rtf')
system = psf.createSystem(params, nonbondedMethod=NoCutoff,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(psf.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(DCDeporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)